# Learning Objectives
In this notebook, you will learn Spark Dataframe APIs.

# Question List

Solve the following questions using Spark Dataframe APIs

### Join

1. easy - https://pgexercises.com/questions/joins/simplejoin.html
2. easy - https://pgexercises.com/questions/joins/simplejoin2.html
3. easy - https://pgexercises.com/questions/joins/self2.html 
4. medium - https://pgexercises.com/questions/joins/threejoin.html (three join)
5. medium - https://pgexercises.com/questions/joins/sub.html (subquery and join)

### Aggregation

1. easy - https://pgexercises.com/questions/aggregates/count3.html Group by order by
2. easy - https://pgexercises.com/questions/aggregates/fachours.html group by order by
3. easy - https://pgexercises.com/questions/aggregates/fachoursbymonth.html group by with condition 
4. easy - https://pgexercises.com/questions/aggregates/fachoursbymonth2.html group by multi col
5. easy - https://pgexercises.com/questions/aggregates/members1.html count distinct
6. med - https://pgexercises.com/questions/aggregates/nbooking.html group by multiple cols, join

### String & Date

1. easy - https://pgexercises.com/questions/string/concat.html format string
2. easy - https://pgexercises.com/questions/string/case.html WHERE + string function
3. easy - https://pgexercises.com/questions/string/reg.html WHERE + string function
4. easy - https://pgexercises.com/questions/string/substr.html group by, substr
5. easy - https://pgexercises.com/questions/date/series.html generate ts
6. easy - https://pgexercises.com/questions/date/bookingspermonth.html extract month from ts

### Question

How can you produce a list of the start times for bookings by members named 'David Farrell'?

https://pgexercises.com/questions/joins/simplejoin.html

In [0]:

bk = spark.sql("select * from booking")
bk.show(5)
mem= spark.sql("SELECT* FROM members")
mem.show(5)
mem1= spark.sql("SELECT* FROM members")
mem1.show(5)
fac= spark.sql("select * from facilities")
fac.show(5)

+------+-----+-----+-------------------+-----+
|bookid|facid|memid|          starttime|slots|
+------+-----+-----+-------------------+-----+
|     0|    3|    1|2012-07-03 11:00:00|    2|
|     1|    4|    1|2012-07-03 08:00:00|    2|
|     2|    6|    0|2012-07-03 18:00:00|    2|
|     3|    7|    1|2012-07-03 19:00:00|    2|
|     4|    8|    1|2012-07-03 10:00:00|    1|
+------+-----+-----+-------------------+-----+
only showing top 5 rows

+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+
|memid| surname|firstname|             address|zipcode|     telephone|recommendedby|           joindate|
+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+
|    0|   GUEST|    GUEST|               GUEST|      0|(000) 000-0000|         null|2012-07-01 00:00:00|
|    1|   Smith|   Darren|8 Bloomsbury Clos...|   4321|  555-555-5555|         null|2012-07-02 12:02:00|
|    2|   Smith|    Tracy|8

### Join

In [0]:
#How can you produce a list of the start times for bookings by members named 'David Farrell'?

david= bk.join(mem, bk.memid== mem.memid, 'inner').select(bk.starttime)
david.filter((mem.surname=='Farrell') & (mem.firstname== 'David')).show()

+-------------------+
|          starttime|
+-------------------+
|2012-09-18 09:00:00|
|2012-09-18 17:30:00|
|2012-09-18 13:30:00|
|2012-09-18 20:00:00|
|2012-09-19 09:30:00|
|2012-09-19 15:00:00|
|2012-09-19 12:00:00|
|2012-09-20 15:30:00|
|2012-09-20 11:30:00|
|2012-09-20 14:00:00|
|2012-09-21 10:30:00|
|2012-09-21 14:00:00|
|2012-09-22 08:30:00|
|2012-09-22 17:00:00|
|2012-09-23 08:30:00|
|2012-09-23 17:30:00|
|2012-09-23 19:00:00|
|2012-09-24 08:00:00|
|2012-09-24 16:30:00|
|2012-09-24 12:30:00|
+-------------------+
only showing top 20 rows



In [0]:
#How can you produce a list of the start times for bookings for tennis courts, for the date '2012-09-21'? Return a list of start time and facility name pairings, ordered by the time.

li= ['Tennis Court 1','Tennis Court 2']
tennis= bk.join(fac, bk.facid== fac.facid, 'inner').select(bk.starttime.alias('Time'),fac.name)
tennis.filter((bk.starttime >='2012-09-21') & (bk.starttime < '2012-09-22') & (fac.name.isin(li))).orderBy(bk.starttime).show()


+-------------------+--------------+
|               Time|          name|
+-------------------+--------------+
|2012-09-21 08:00:00|Tennis Court 1|
|2012-09-21 08:00:00|Tennis Court 2|
|2012-09-21 09:30:00|Tennis Court 1|
|2012-09-21 10:00:00|Tennis Court 2|
|2012-09-21 11:30:00|Tennis Court 2|
|2012-09-21 12:00:00|Tennis Court 1|
|2012-09-21 13:30:00|Tennis Court 1|
|2012-09-21 14:00:00|Tennis Court 2|
|2012-09-21 15:30:00|Tennis Court 1|
|2012-09-21 16:00:00|Tennis Court 2|
|2012-09-21 17:00:00|Tennis Court 1|
|2012-09-21 18:00:00|Tennis Court 2|
+-------------------+--------------+



In [0]:
#How can you output a list of all members, including the individual who recommended them (if any)? Ensure that results are ordered by (surname, firstname).

mem.join(mem1, mem1.memid==mem.recommendedby, 'left').select(mem.firstname.alias('memfname'),mem.surname.alias('memsname'),mem1.firstname.alias('recfname'),mem1.surname.alias('recsname')).orderBy('memsname','memfname').show()

+---------+---------+---------+--------+
| memfname| memsname| recfname|recsname|
+---------+---------+---------+--------+
| Florence|    Bader|   Ponder|Stibbons|
|     Anne|    Baker|   Ponder|Stibbons|
|  Timothy|    Baker|   Jemima| Farrell|
|      Tim|   Boothe|      Tim|  Rownam|
|   Gerald|  Butters|   Darren|   Smith|
|     Joan|   Coplin|  Timothy|   Baker|
|    Erica|  Crumpet|    Tracy|   Smith|
|    Nancy|     Dare|   Janice|Joplette|
|    David|  Farrell|     null|    null|
|   Jemima|  Farrell|     null|    null|
|    GUEST|    GUEST|     null|    null|
|  Matthew|  Genting|   Gerald| Butters|
|     John|     Hunt|Millicent| Purview|
|    David|    Jones|   Janice|Joplette|
|  Douglas|    Jones|    David|   Jones|
|   Janice| Joplette|   Darren|   Smith|
|     Anna|Mackenzie|   Darren|   Smith|
|  Charles|     Owen|   Darren|   Smith|
|    David|   Pinker|   Jemima| Farrell|
|Millicent|  Purview|    Tracy|   Smith|
+---------+---------+---------+--------+
only showing top

In [0]:
#How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name followed by the facility name.

from pyspark.sql.functions import concat,concat_ws,asc,desc

li= ['Tennis Court 1','Tennis Court 2']

data = mem.join(bk, mem.memid==bk.memid, 'inner')\
.join(fac, bk.facid==fac.facid, 'inner')

data1= data.select(concat_ws(' ',mem.firstname, mem.surname).alias('Name'),fac.name.alias('facility')).filter((fac.name.isin(li)))

data1.distinct().orderBy('Name','facility').show()

+--------------+--------------+
|          Name|      facility|
+--------------+--------------+
|    Anne Baker|Tennis Court 1|
|    Anne Baker|Tennis Court 2|
|  Burton Tracy|Tennis Court 1|
|  Burton Tracy|Tennis Court 2|
|  Charles Owen|Tennis Court 1|
|  Charles Owen|Tennis Court 2|
|  Darren Smith|Tennis Court 2|
| David Farrell|Tennis Court 1|
| David Farrell|Tennis Court 2|
|   David Jones|Tennis Court 1|
|   David Jones|Tennis Court 2|
|  David Pinker|Tennis Court 1|
| Douglas Jones|Tennis Court 1|
| Erica Crumpet|Tennis Court 1|
|Florence Bader|Tennis Court 1|
|Florence Bader|Tennis Court 2|
|   GUEST GUEST|Tennis Court 1|
|   GUEST GUEST|Tennis Court 2|
|Gerald Butters|Tennis Court 1|
|Gerald Butters|Tennis Court 2|
+--------------+--------------+
only showing top 20 rows



In [0]:
#SQL version for reference

data1= spark.sql("select distinct m.firstname ||' '|| m.surname as member, f.name as facility from  members m inner join booking b on \
                 m.memid=b.memid inner join facilities f on b.facid =f.facid where f.name in ('Tennis Court 1','Tennis Court 2') order by \
                 member,facility asc")
data1.distinct().show()  

+--------------+--------------+
|        member|      facility|
+--------------+--------------+
|    Anne Baker|Tennis Court 1|
|    Anne Baker|Tennis Court 2|
|  Burton Tracy|Tennis Court 1|
|  Burton Tracy|Tennis Court 2|
|  Charles Owen|Tennis Court 1|
|  Charles Owen|Tennis Court 2|
|  Darren Smith|Tennis Court 2|
| David Farrell|Tennis Court 1|
| David Farrell|Tennis Court 2|
|   David Jones|Tennis Court 1|
|   David Jones|Tennis Court 2|
|  David Pinker|Tennis Court 1|
| Douglas Jones|Tennis Court 1|
| Erica Crumpet|Tennis Court 1|
|Florence Bader|Tennis Court 1|
|Florence Bader|Tennis Court 2|
|   GUEST GUEST|Tennis Court 1|
|   GUEST GUEST|Tennis Court 2|
|Gerald Butters|Tennis Court 1|
|Gerald Butters|Tennis Court 2|
+--------------+--------------+
only showing top 20 rows



In [0]:
#How can you output a list of all members, including the individual who recommended them (if any), without using any joins? Ensure that there are no duplicates in the list, and that each firstname + surname pairing is formatted as a column and ordered.
from pyspark.sql.functions import concat,concat_ws,asc,desc

mem.join(mem1, mem1.memid==mem.recommendedby, 'left')\
    .select(concat_ws(' ',mem.firstname,mem.surname).alias('member'),concat_ws(' ',mem1.firstname,mem1.surname).alias('recommender')).orderBy('member').show()

+--------------------+---------------+
|              member|    recommender|
+--------------------+---------------+
|      Anna Mackenzie|   Darren Smith|
|          Anne Baker|Ponder Stibbons|
|        Burton Tracy|               |
|        Charles Owen|   Darren Smith|
|        Darren Smith|               |
|        Darren Smith|               |
|       David Farrell|               |
|         David Jones|Janice Joplette|
|        David Pinker| Jemima Farrell|
|       Douglas Jones|    David Jones|
|       Erica Crumpet|    Tracy Smith|
|      Florence Bader|Ponder Stibbons|
|         GUEST GUEST|               |
|      Gerald Butters|   Darren Smith|
|    Henrietta Rumney|Matthew Genting|
|Henry Worthington...|    Tracy Smith|
| Hyacinth Tupperware|               |
|          Jack Smith|   Darren Smith|
|     Janice Joplette|   Darren Smith|
|      Jemima Farrell|               |
+--------------------+---------------+
only showing top 20 rows



###Aggregation

In [0]:

bk = spark.sql("select * from booking")
bk.show(5)
mem= spark.sql("SELECT* FROM members")
mem.show(5)
mem1= spark.sql("SELECT* FROM members")
mem1.show(5)
fac= spark.sql("select * from facilities")
fac.show(5)

+------+-----+-----+-------------------+-----+
|bookid|facid|memid|          starttime|slots|
+------+-----+-----+-------------------+-----+
|     0|    3|    1|2012-07-03 11:00:00|    2|
|     1|    4|    1|2012-07-03 08:00:00|    2|
|     2|    6|    0|2012-07-03 18:00:00|    2|
|     3|    7|    1|2012-07-03 19:00:00|    2|
|     4|    8|    1|2012-07-03 10:00:00|    1|
+------+-----+-----+-------------------+-----+
only showing top 5 rows

+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+
|memid| surname|firstname|             address|zipcode|     telephone|recommendedby|           joindate|
+-----+--------+---------+--------------------+-------+--------------+-------------+-------------------+
|    0|   GUEST|    GUEST|               GUEST|      0|(000) 000-0000|         null|2012-07-01 00:00:00|
|    1|   Smith|   Darren|8 Bloomsbury Clos...|   4321|  555-555-5555|         null|2012-07-02 12:02:00|
|    2|   Smith|    Tracy|8

In [0]:
#Produce a count of the number of recommendations each member has made. Order by member ID.

rec= mem.groupBy("recommendedby").count().orderBy('recommendedby')
rec.filter(mem.recommendedby > 0).show()

+-------------+-----+
|recommendedby|count|
+-------------+-----+
|            1|    5|
|            2|    3|
|            3|    1|
|            4|    2|
|            5|    1|
|            6|    1|
|            9|    2|
|           11|    1|
|           13|    2|
|           15|    1|
|           16|    1|
|           20|    1|
|           30|    1|
+-------------+-----+



In [0]:
#Produce a list of the total number of slots booked per facility. For now, just produce an output table consisting of facility id and slots, sorted by facility id.

bk.groupBy('facid').sum('slots').orderBy('facid').show()


+-----+----------+
|facid|sum(slots)|
+-----+----------+
|    0|      1320|
|    1|      1278|
|    2|      1209|
|    3|       830|
|    4|      1404|
|    5|       228|
|    6|      1104|
|    7|       908|
|    8|       911|
+-----+----------+



In [0]:
#Produce a list of the total number of slots booked per facility in the month of September 2012. Produce an output table consisting of facility id and slots, sorted by the number of slots.
# 
from pyspark.sql.functions import col
septbk= bk.filter(
    (col("starttime") >= "2012-09-01") & (col("starttime") < "2012-10-01"))
septbk.groupBy('facid').sum('slots').show()


+-----+----------+
|facid|sum(slots)|
+-----+----------+
|    1|       588|
|    6|       540|
|    3|       422|
|    5|       122|
|    4|       648|
|    8|       471|
|    7|       426|
|    2|       570|
|    0|       591|
+-----+----------+



In [0]:
#SQL for reference 
# 
data1= spark.sql ("select facid, sum(slots) from booking where starttime >= '2012-09-01'	and starttime < '2012-10-01'\
    	group by facid")
data1.show(5)

+-----+----------+
|facid|sum(slots)|
+-----+----------+
|    1|       588|
|    6|       540|
|    3|       422|
|    5|       122|
|    4|       648|
+-----+----------+
only showing top 5 rows



In [0]:
#Produce a list of the total number of slots booked per facility per month in the year of 2012. Produce an output table consisting of facility id and slots, sorted by the id and month.

from pyspark.sql.functions import year, month, col

bwym = bk.withColumn("year", year("starttime")) \
        .withColumn("month", month("starttime"))
bwym.filter(col('year') == 2012).groupBy('facid','month').sum('slots').orderBy('facid','month').show()

+-----+-----+----------+
|facid|month|sum(slots)|
+-----+-----+----------+
|    0|    7|       270|
|    0|    8|       459|
|    0|    9|       591|
|    1|    7|       207|
|    1|    8|       483|
|    1|    9|       588|
|    2|    7|       180|
|    2|    8|       459|
|    2|    9|       570|
|    3|    7|       104|
|    3|    8|       304|
|    3|    9|       422|
|    4|    7|       264|
|    4|    8|       492|
|    4|    9|       648|
|    5|    7|        24|
|    5|    8|        82|
|    5|    9|       122|
|    6|    7|       164|
|    6|    8|       400|
+-----+-----+----------+
only showing top 20 rows



In [0]:
#Find the total number of members (including guests) who have made at least one booking.

from pyspark.sql.functions import count,col

distinct_member_bookings = bk.groupBy("memid").agg(count("*").alias("count"))

# Filter for members with at least one booking
members_with_bookings = distinct_member_bookings.where(col("count") >= 1)

# Get the total count of members with bookings
total_members_with_bookings = members_with_bookings.count()

print("Total members with at least one booking:", total_members_with_bookings)


Total members with at least one booking: 30


In [0]:
#Produce a list of each member name, id, and their first booking after September 1st 2012. Order by member ID.
from pyspark.sql.functions import min,max,col

data = mem.join(bk, mem.memid==bk.memid, 'inner').select(mem.surname,mem.firstname, mem.memid, bk.starttime).filter(bk.starttime >= '2012-09-01')

data.groupBy('surname','firstname','memid').agg(min("starttime")).orderBy('memid').show()

+---------+---------+-----+-------------------+
|  surname|firstname|memid|     min(starttime)|
+---------+---------+-----+-------------------+
|    GUEST|    GUEST|    0|2012-09-01 08:00:00|
|    Smith|   Darren|    1|2012-09-01 09:00:00|
|    Smith|    Tracy|    2|2012-09-01 11:30:00|
|   Rownam|      Tim|    3|2012-09-01 16:00:00|
| Joplette|   Janice|    4|2012-09-01 15:00:00|
|  Butters|   Gerald|    5|2012-09-02 12:30:00|
|    Tracy|   Burton|    6|2012-09-01 15:00:00|
|     Dare|    Nancy|    7|2012-09-01 12:30:00|
|   Boothe|      Tim|    8|2012-09-01 08:30:00|
| Stibbons|   Ponder|    9|2012-09-01 11:00:00|
|     Owen|  Charles|   10|2012-09-01 11:00:00|
|    Jones|    David|   11|2012-09-01 09:30:00|
|    Baker|     Anne|   12|2012-09-01 14:30:00|
|  Farrell|   Jemima|   13|2012-09-01 09:30:00|
|    Smith|     Jack|   14|2012-09-01 11:00:00|
|    Bader| Florence|   15|2012-09-01 10:30:00|
|    Baker|  Timothy|   16|2012-09-01 15:00:00|
|   Pinker|    David|   17|2012-09-01 08

###String & Date

In [0]:
#Output the names of all members, formatted as 'Surname, Firstname'

from pyspark.sql.functions import concat,concat_ws,asc,desc

mem.select(concat_ws(',',mem.surname,mem.firstname).alias('name')).show()

+---------------+
|           name|
+---------------+
|    GUEST,GUEST|
|   Smith,Darren|
|    Smith,Tracy|
|     Rownam,Tim|
|Joplette,Janice|
| Butters,Gerald|
|   Tracy,Burton|
|     Dare,Nancy|
|     Boothe,Tim|
|Stibbons,Ponder|
|   Owen,Charles|
|    Jones,David|
|     Baker,Anne|
| Farrell,Jemima|
|     Smith,Jack|
| Bader,Florence|
|  Baker,Timothy|
|   Pinker,David|
|Genting,Matthew|
| Mackenzie,Anna|
+---------------+
only showing top 20 rows



In [0]:
#Perform a case-insensitive search to find all facilities whose name begins with 'tennis'. Retrieve all columns.
from pyspark.sql.functions import upper,lower,col

filtered_facilities = fac.filter(fac.name.startswith("Tennis"))
filtered_facilities.show()


+-----+--------------+----------+---------+-------------+------------------+
|facid|          name|membercost|guestcost|initialoutlay|monthlymaintenance|
+-----+--------------+----------+---------+-------------+------------------+
|    0|Tennis Court 1|       5.0|     25.0|        10000|               200|
|    1|Tennis Court 2|       5.0|     25.0|         8000|               200|
+-----+--------------+----------+---------+-------------+------------------+



In [0]:
#You've noticed that the club's member table has telephone numbers with very inconsistent formatting. You'd like to find all the telephone numbers that contain parentheses, returning the member ID and telephone number sorted by member ID.

from pyspark.sql.functions import col, regexp_extract


# Define a regular expression to extract telephone numbers with parentheses
telephone_pattern = r'\((\d+)\)'

# Extract telephone numbers using regexp_extract and create a new column 'telephone_number'
members_with_telephone = mem.withColumn(
    "telephone_number",regexp_extract(col("telephone"), telephone_pattern, 1))
    

# Filter rows where telephone numbers were found
members_with_parentheses = members_with_telephone.filter(col("telephone_number") != "")

    # Select and order by member ID and telephone number
result_df = members_with_parentheses.select("memid", "telephone").orderBy("memid")

    # Show the result
result_df.show()



+-----+--------------+
|memid|     telephone|
+-----+--------------+
|    0|(000) 000-0000|
|    3|(844) 693-0723|
|    4|(833) 942-4710|
|    5|(844) 078-4130|
|    6|(822) 354-9973|
|    7|(833) 776-4001|
|    8|(811) 433-2547|
|    9|(833) 160-3900|
|   10|(855) 542-5251|
|   11|(844) 536-8036|
|   13|(855) 016-0163|
|   14|(822) 163-3254|
|   15|(833) 499-3527|
|   20|(811) 972-1377|
|   21|(822) 661-2898|
|   22|(822) 499-2232|
|   24|(822) 413-1470|
|   27|(822) 989-8876|
|   28|(855) 755-9876|
|   29|(855) 894-3758|
+-----+--------------+
only showing top 20 rows



In [0]:
#You'd like to produce a count of how many members you have whose surname starts with each letter of the alphabet. Sort by the letter, and don't worry about printing out a letter if the count is 0.


from pyspark.sql.functions import substring,count

data= mem.withColumn('letter', substring('surname', 1,1))

data.groupBy('letter').agg(count(data.letter)).orderBy(data.letter).show()

+------+-------------+
|letter|count(letter)|
+------+-------------+
|     B|            5|
|     C|            2|
|     D|            1|
|     F|            2|
|     G|            2|
|     H|            1|
|     J|            3|
|     M|            1|
|     O|            1|
|     P|            2|
|     R|            2|
|     S|            6|
|     T|            2|
|     W|            1|
+------+-------------+



In [0]:
#Produce a list of all the dates in October 2012. They can be output as a timestamp (with time set to midnight) or a date.
from pyspark.sql.functions import expr
# Create a DataFrame with a single column containing the dates in October 2012
dates_df = spark.range(1, 32).select(expr("'2012-10-' || id as date"))

# Show the list of dates
dates_df. select(expr("cast(date as timestamp) as timestampe")).show()

+-------------------+
|         timestampe|
+-------------------+
|2012-10-01 00:00:00|
|2012-10-02 00:00:00|
|2012-10-03 00:00:00|
|2012-10-04 00:00:00|
|2012-10-05 00:00:00|
|2012-10-06 00:00:00|
|2012-10-07 00:00:00|
|2012-10-08 00:00:00|
|2012-10-09 00:00:00|
|2012-10-10 00:00:00|
|2012-10-11 00:00:00|
|2012-10-12 00:00:00|
|2012-10-13 00:00:00|
|2012-10-14 00:00:00|
|2012-10-15 00:00:00|
|2012-10-16 00:00:00|
|2012-10-17 00:00:00|
|2012-10-18 00:00:00|
|2012-10-19 00:00:00|
|2012-10-20 00:00:00|
+-------------------+
only showing top 20 rows



In [0]:
#Return a count of bookings for each month, sorted by month

bwym = bk.withColumn("month", month("starttime"))

bwym.groupBy('month').count().orderBy('month').show()

+-----+-----+
|month|count|
+-----+-----+
|    1|    1|
|    7|  658|
|    8| 1472|
|    9| 1913|
+-----+-----+

